In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import librosa
import numpy as np
import os
import glob
import json

import torch
from text import text_to_sequence, cmudict
from text.symbols import symbols
import commons
import attentions
import modules
import models
import utils
import shutil
from tqdm import tqdm

In [2]:
model_dir = "logs/base_blank_emo_lang"
hps = utils.get_hparams_from_dir(model_dir)

In [3]:
stft_fun = commons.TacotronSTFT(
            hps.data.filter_length, hps.data.hop_length, hps.data.win_length,
            hps.data.n_mel_channels, hps.data.sampling_rate, hps.data.mel_fmin,
            hps.data.mel_fmax)

In [6]:
files_array = glob.glob("/run/media/viblab/Markov2/Pras/Thesis/Database/VIBID/wavs/**/*.wav")

In [9]:
for file in tqdm(files_array):
    audio, sampling_rate = utils.load_wav_to_torch(file)
    if sampling_rate != stft_fun.sampling_rate:
        raise ValueError("{} {} SR doesn't match target {} SR".format(
            sampling_rate, stft_fun.sampling_rate))
    audio = audio + torch.rand_like(audio)
    audio_norm = audio / hps.data.max_wav_value
    audio_norm = audio_norm.unsqueeze(0)
    try:
        melspec = stft_fun.mel_spectrogram(audio_norm)
        melspec = torch.squeeze(melspec, 0)

        filename = file.split("/")[-1].split(".")[0]
        np.save(f"/run/media/viblab/Markov2/Pras/Thesis/TryMyOwn/hifi-gan/ft_dataset/{filename}.npy", melspec.cpu().detach().numpy())
        shutil.copy(file, f"/run/media/viblab/Markov2/Pras/Thesis/TryMyOwn/hifi-gan/ft_dataset/{filename}.wav")
    except:
        pass

100%|██████████| 4367/4367 [00:33<00:00, 128.64it/s]
